# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import scipy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.special import gamma, digamma, polygamma
import math

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import random

import scipy.optimize
from scipy.stats import ttest_ind
from scipy.optimize import Bounds
from sklearn.preprocessing import StandardScaler

from prettytable import PrettyTable

In [3]:
import multiprocessing

In [4]:
multiprocessing.cpu_count()

8

In [5]:
import dirichlet_regression

In [6]:
from scipy.optimize import fmin, newton, minimize

In [77]:
# sur mon ordi perso:
scipy.__version__

'1.4.1'

# Loading Dataset

In [7]:
Y_occitanie = pd.read_csv('Data Dirichlet/occitanie/Y_occitanie.csv', sep=';')

In [8]:
X_occitanie = pd.read_csv('Data Dirichlet/occitanie/X_occitanie.csv', sep=';')

In [9]:
X_occitanie = X_occitanie.iloc[:,1:]
X_occitanie = X_occitanie.iloc[:,:-1]

In [12]:
coordinates = pd.read_csv('Data Dirichlet/occitanie/coordinates_cendroids.csv', sep=';')

In [14]:
distance_matrix = scipy.spatial.distance_matrix(coordinates,coordinates)

In [63]:
W = np.copy(distance_matrix)
W[W > 35000] = 0

In [64]:
np.mean([np.sum(W[i]!=0) for i in range(207)])

12.096618357487923

In [65]:
# inverse distance
W[W>0] = 1/W[W>0]

# row-normalize
W = W/W.sum(axis=1)[:,None]

In [102]:
W_occitanie = pd.read_csv('Data Dirichlet/occitanie/W_elections_5nn.csv', sep=' ', header=None)

In [16]:
W_occitanie_10 = pd.read_csv('Data Dirichlet/occitanie/W_elections_10nn.csv', sep=' ', header=None)

In [ ]:
W = np.array(W_occitanie)

# Performing the Dirichlet regression

In [66]:
X = np.array(X_occitanie)
Y = np.array(Y_occitanie)

In [67]:
Z = np.ones((207,1))
gamma_0 = [0.]

In [68]:
X = StandardScaler().fit(X).transform(X)

In [140]:
%%time
dirichRegressor = dirichlet_regression.dirichletRegressor()
dirichRegressor.fit(X, Y, parametrization='common')

Optimization terminated successfully.
Wall time: 987 ms


In [141]:
r2_score(Y,dirichRegressor.mu)

0.008785738921756122

In [142]:
%%time
dirichRegressor = dirichlet_regression.dirichletRegressor()
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z)

Optimization terminated successfully.
Wall time: 893 ms


In [143]:
r2_score(Y,dirichRegressor.mu)

0.4872304823343588

In [136]:
%%time
dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True, maxiter=2000, maxfun=20000)
dirichRegressor.fit(X, Y, W=W, parametrization='common')

b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
Wall time: 19.3 s


In [137]:
r2_score(Y,dirichRegressor.mu)

0.05704851915593345

In [138]:
%%time
dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True, maxiter=2000, maxfun=20000)
dirichRegressor.fit(X, Y, W=W, parametrization='alternative', gamma_0=gamma_0, Z=Z)

b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
Wall time: 19.7 s


In [139]:
r2_score(Y,dirichRegressor.mu)

0.5751095610093545

In [71]:
%%time
dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True, maxiter=2000, maxfun=50000)
dirichRegressor.fit(X, Y, W=W, parametrization='alternative', gamma_0=gamma_0, Z=Z)

b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
Wall time: 59.6 s


In [72]:
r2_score(Y,dirichRegressor.mu)

0.5977060374467865